In [ ]:
#python3 t2_gleasondata.py

In [8]:
import os, gc, subprocess, time, sys, shutil

import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [9]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [11]:
from panda_bvv_config import *

## 1. Inputs ##

In [12]:
Zoom = 2
input_folder = train_size2
#input_folder = test_size2
input_mask_folder = mask_size2
new_proc_folder = gleason_path
kw_for_tile_func = {'gauss_blur':20}

In [13]:
# wrapping inputs:
if not os.path.exists(new_proc_folder):
    print("[INFO] 'creating {}' directory".format(new_proc_folder))
    os.makedirs(new_proc_folder)


[INFO] 'creating /run/media/admin/kagg/panda/gs' directory


In [14]:
for grade in gs_classes:
    if not os.path.exists(os.path.join(gleason_path, f'gs{grade}')):
        print("[INFO] 'creating {}' directory".format(os.path.join(gleason_path, f'gs{grade}')))
        os.makedirs(os.path.join(gleason_path, f'gs{grade}'))


[INFO] 'creating /run/media/admin/kagg/panda/gs/gs0' directory
[INFO] 'creating /run/media/admin/kagg/panda/gs/gs3' directory
[INFO] 'creating /run/media/admin/kagg/panda/gs/gs4' directory
[INFO] 'creating /run/media/admin/kagg/panda/gs/gs5' directory


## 2. Standard functions ##

In [15]:
#we write to foldersbased on gleason score
def datanew_wr_gleason(output_data_path_, 
                       img_num_, 
                       data_new_,
                       gleason_class):
    wr_to = os.path.join(output_data_path_, 'gs' + gleason_class, img_num_ + '.png')    
    cv2.imwrite(wr_to, data_new_)


In [16]:
#wrapper for image processing function
def process_all_images(proc_func):
    
    def wrapper(zoom = Zoom,
                input_biopsy_folder = input_folder,
                output_data_path = new_proc_folder,
                df_name = gleason_labels,
                tif_file = False,
                with_mask = True,
                input_mask_folder = input_mask_folder,
                **kw):
        print(input_biopsy_folder)
        print(output_data_path)
        with tqdm(total=df_name.shape[0]) as pbar:
            for  i, row in enumerate(tqdm(df_name.iterrows())):
                img_num = row[1]['image_id']
                provider = row[1]['data_provider']                         
                mask = None
                if tif_file:
                    try:
                        biopsy = skimage.io.MultiImage(os.path.join(input_biopsy_folder,
                                                                    img_num + '.tiff'))[zoom]               
                    except:
                        print('Failed to read tiff:', img_num)                        
                else:
                    try:
                        biopsy = skimage.io.imread(os.path.join(input_biopsy_folder, img_num + '.png'))
                        shape = biopsy.shape
                        if shape[0]<shape[1]: 
                            biopsy = np.rot90(biopsy)
                            shape = biopsy.shape 
                    except:
                        print(f'can not proceed with {img_num}')   
                if with_mask:
                    try:
                        mask = skimage.io.imread(os.path.join(input_mask_folder, img_num + '_mask.png'))
                        shape = mask.shape
                        if shape[0]<shape[1]: 
                            mask = np.rot90(mask)
                            shape = mask.shape 
                    except:
                        print('Failed to process mask:', img_num) 
                try: 
                    data_new = proc_func(biopsy, 
                                         mask, 
                                         provider,
                                         row[1]['gleason_score'],
                                         **kw)
                    datanew_wr_gleason(output_data_path_= output_data_path,
                                       img_num_ = img_num, 
                                       data_new_ = data_new,
                                       gleason_class = str(row[1]['gleason_score']))                    
                                      
                except Exception as ee:
                    print('Processing mistake:\n', ee, '\n', img_num)            
                try:
                    del biopsy, mask, data_new, mask_new
                except:
                    pass
                pbar.update(1)
            gc.collect()
        return
    
    return wrapper

In [17]:
def change_karolinska_mask_for_isup_grade_direct(mask_data, chan):
    """
    This function returns pre-processed radbound mask for image multiplication
    """

    try:
        new_mask = np.zeros(mask_data.shape)#.astype('uint8')
        cancerous_tissue = False
        try:             
            mask_area = np.where(mask_data[:,:,chan] == 2)
            new_mask[mask_area[0], mask_area[1], :] = [1, 1, 1] 
            del mask_area, mask_data
            if new_mask.any() >0: 
                cancerous_tissue = True    
        except Exception as ee:
            print("Mask {%s} processing mistake: " %mask_image, ee)
    except Exception as ee:
        print("Mask {%s} read mistake: "  %mask_image, str(ee))

    gc.collect()                   
    
    return cancerous_tissue, new_mask

In [18]:
def change_radboud_mask_for_gleason_grade_direct(mask_data, 
                                                 chan,
                                                 gleason_class,
                                                 gauss_bl = 10):
    """
    This function returns pre-processed radbound mask for image multiplication
    """
    cancerous_tissue = False
    if gleason_class == 0: 
        new_mask = mask_data
        
    else:
        try:
            new_mask = np.zeros(mask_data.shape)#.astype('uint8')
            try: 
                mask_area = np.where(mask_data[:,:,chan] == gleason_class)
                new_mask[mask_area[0], mask_area[1], :] = [1, 1, 1]
                del mask_area
                if new_mask.any() >0: 
                    cancerous_tissue = True
                del mask_data,    
            except Exception as ee:
                print("Mask {%s} processing mistake: " %mask_image, ee)
        except Exception as ee:
            print("Mask {%s} read mistake: "  %mask_image, str(ee))

        #new_mask[:,:,:] = cv2.blur(new_mask[:,:, :],(gauss_bl,gauss_bl))
        new_mask[:,:,:] = cv2.bilateralFilter(np.float32(new_mask[:,:,:]),gauss_bl,25,25)
        #medianBlur(new_mask[:,:, 2],5)
        #new_mask[:,:,:] = cv2.GaussianBlur(new_mask[:,:,:],(gauss_bl,gauss_bl), 0)
        #new_mask[:,:,:] =  np.rint(new_mask[:,:,:]) #.astype(int) 
        new_mask[:,:,:] =  np.ceil(new_mask[:,:,:]) #.astype(int) 
        #gc.collect()                   
    
    return cancerous_tissue, new_mask

In [19]:
def get_cancer_area_from_mask(cancer_mask, biopsy):
    """
    Returns multiplication of mask and image
    """
    try:
        only_cancer = np.multiply(cancer_mask,biopsy).astype(int)
        #cv2.imwrite(save_path, only_cancer, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
        #del cancer_mask, biopsy #, only_cancer
    except Exception as ee:
        print(str(ee))
    
    return only_cancer  

## 3. Processing functions ##

In [20]:
@process_all_images
def get_cancer_area_for_gleason_grade(biopsy, 
                                   mask, 
                                   provider, 
                                   gleason_class, 
                                   **kw):
    """
    Extracts cancer area only (if cancerous), from image and mask; or unchanged (if non-cancerous).
    Split cancer areas on Gleason.
    Do NOT tile image
    """

    gauss_bl = kw.get('gauss_bl') if kw.get('gauss_bl') else 20    
    channel = 0 if len(np.unique(mask[:,:,0])) >1 else 2    

    if provider == 'karolinska':
        cancerous, new_mask  = change_karolinska_mask_for_isup_grade_direct(mask, chan = channel)
    elif provider == 'radboud':
        cancerous, new_mask  = change_radboud_mask_for_gleason_grade_direct(mask, 
                                                                            chan = channel,
                                                                            gleason_class=gleason_class,
                                                                            gauss_bl = gauss_bl)

    if cancerous:
        temp_im = get_cancer_area_from_mask(new_mask, biopsy)
        data_new = cv2.bitwise_not(temp_im.astype('uint8') * 255)
        #inv = cv2.bitwise_not(temp_im )
        #inv = biopsy
    else:
        data_new = biopsy
    
    #data_new = cv2.bitwise_not(inv)
    return data_new

## 4. Process input files for training purpose ##

In [21]:
kw_for_tile_func = {'gauss_blur':20}
get_cancer_area_for_gleason_grade(df_name = test_gleason_labels,
                                  with_mask = True,
                                  **kw_for_tile_func)

  0%|          | 0/1348 [00:00<?, ?it/s]
0it [00:00, ?it/s]

/run/media/admin/kagg/panda/test_size2
/run/media/admin/kagg/panda/gs


 20%|█▉        | 266/1348 [02:37<07:35,  2.37it/s]
266it [02:37,  2.38it/s]

Failed to process mask: c3ca4a75be0055aefe79f6849804d62c
Processing mistake:
 'NoneType' object is not subscriptable 
 c3ca4a75be0055aefe79f6849804d62c


 25%|██▌       | 340/1348 [03:17<07:39,  2.19it/s]
340it [03:17,  2.19it/s]

...

 80%|███████▉  | 1073/1348 [09:58<03:45,  1.22it/s]
1073it [09:58,  1.22it/s]

Failed to process mask: 6df7a3d8b4955d162fcc11bce003860a
Processing mistake:
 'NoneType' object is not subscriptable 
 6df7a3d8b4955d162fcc11bce003860a


 89%|████████▉ | 1205/1348 [11:31<01:11,  2.00it/s]
1205it [11:31,  2.00it/s]

Failed to process mask: 0ebea13245a2b04ca3e6a482091e35d6
Processing mistake:
 'NoneType' object is not subscriptable 
 0ebea13245a2b04ca3e6a482091e35d6


100%|██████████| 1348/1348 [13:08<00:00,  1.72it/s]
1348it [13:08,  1.71it/s]
100%|██████████| 1348/1348 [13:09<00:00,  1.71it/s]


In [1]:
import os
module_name = 't2_gleasondata'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)